## MENGUJI MANUAL PREDICTOR

In [ ]:
import pandas as pd
import os
from sklearn.datasets import make_regression
from noventis.predictor import ManualPredictor

def test_manual_regression():
    """
    Fungsi untuk menguji ManualPredictor pada tugas regresi.
    Menggunakan data sintetis.
    """
    print("🚀 MEMULAI PENGUJIAN: MANUAL PREDICTOR UNTUK REGRESI")
    print("-" * 60)

    # --- 1. Persiapan Data dan Direktori ---
    output_dir = "noventis_output/regression_manual"
    os.makedirs(output_dir, exist_ok=True)
    print(f"Direktori output disiapkan di: {output_dir}")

    # Membuat data regresi sintetis
    X, y = make_regression(n_samples=200, n_features=10, noise=25, random_state=42)
    df_regr = pd.DataFrame(X, columns=[f'feature_{i}' for i in range(10)])
    df_regr['target'] = y
    print("✅ Dataset regresi sintetis berhasil dibuat.")
    print("Data head:")
    print(df_regr.head())
    print("-" * 60)

    # --- 2. Menjalankan Pipeline dengan Satu Model ---
    print("\n--- Menjalankan ManualPredictor (Single Model: XGBoost) ---")
    manual_predictor_single = ManualPredictor(model_name='xgboost', task='regression')

    # Jalankan pipeline
    result_single = manual_predictor_single.run_pipeline(df=df_regr, target_column='target')
    print(f"Model: {result_single['model_name']}, R2-Score: {result_single['metrics']['r2_score']:.4f}")

    # Simpan model
    model_path = os.path.join(output_dir, "best_xgboost_model.pkl")
    manual_predictor_single.save_model(model_path)
    print("-" * 60)

    # --- 3. Menjalankan Pipeline dengan Perbandingan Beberapa Model ---
    print("\n--- Menjalankan ManualPredictor (Multiple Models Comparison) ---")
    manual_predictor_multi = ManualPredictor(
        model_name=['linear_regression', 'random_forest', 'lightgbm'],
        task='regression'
    )
    result_multi = manual_predictor_multi.run_pipeline(df=df_regr, target_column='target')

    print("\n--- Hasil Perbandingan Manual ---")
    best_model_details = result_multi['best_model_details']
    print(f"Model terbaik: {best_model_details['model_name']}, R2-Score: {best_model_details['metrics']['r2_score']:.4f}")
    print("-" * 60)
    
    print("\n🎉 Pengujian Regresi Selesai.")

if __name__ == '__main__':
    test_manual_regression()

## MENGUJI AUTO PREDICTOR

In [ ]:
# examples/test_automl.py

import pandas as pd
import os
from sklearn.datasets import load_iris
from noventis.predictor import NoventisAutoML

def test_automl_classification():
    """
    Fungsi untuk menguji NoventisAutoML pada tugas klasifikasi.
    Menggunakan dataset Iris.
    """
    print("🚀 MEMULAI PENGUJIAN: AutoML UNTUK KLASIFIKASI (DATASET IRIS)")
    print("-" * 60)

    # --- 1. Persiapan Data dan Direktori ---
    output_dir = "noventis_output/iris_automl"
    os.makedirs(output_dir, exist_ok=True)
    print(f"Direktori output disiapkan di: {output_dir}")

    # Memuat data Iris
    iris = load_iris(as_frame=True)
    df_iris = iris.frame
    # Mengganti nama kolom target agar lebih deskriptif
    df_iris.rename(columns={'target': 'species_target'}, inplace=True)
    print("✅ Dataset Iris berhasil dimuat.")
    print("Data head:")
    print(df_iris.head())
    print("-" * 60)

    # --- 2. Menjalankan Proses AutoML ---
    print("\n--- Menjalankan AutoML Fit (time_budget=30s) ---")
    # Inisialisasi NoventisAutoML
    automl_predictor = NoventisAutoML(data=df_iris, target='species_target')

    # Menjalankan proses fit dengan time budget 30 detik
    automl_results = automl_predictor.fit(
        time_budget=30,
        explain=True, # Membuat visualisasi otomatis
        output=f"{output_dir}/automl_model.pkl" # Menyimpan model terbaik
    )
    print("\n--- Hasil AutoML ---")
    print(f"Estimator terbaik yang ditemukan: {automl_results['best_estimator']}")
    print(f"Metrik F1-Score Macro: {automl_results['metrics']['f1_score']:.4f}")
    print("-" * 60)

    # --- 3. Membandingkan Model ---
    print("\n--- Menjalankan Perbandingan Model Manual ---")
    comparison_results = automl_predictor.compare_models(
        models_to_compare=['logistic_regression', 'random_forest']
    )
    print("\n--- Hasil Perbandingan ---")
    print(pd.DataFrame(comparison_results['rankings']))
    print("-" * 60)

    print("\n🎉 Pengujian Klasifikasi Selesai. Cek folder 'noventis_output' untuk melihat visualisasi.")

if __name__ == '__main__':
    test_automl_classification()
